In [1]:
import pandas as pd
import numpy as np

from os.path import join
from typing import List, Dict

from utils import missing_zero_values_table
from constants import data_folder

In [2]:
raw_data: pd.DataFrame = pd.read_csv(join(data_folder, "Provisional_COVID-19_Death_Counts_by_Sex__Age__and_State.csv"))
raw_data.head()

,Data as of,Start week,End Week,State,Sex,Age group,COVID-19 Deaths,Total Deaths,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths",Footnote
0,12/02/2020,02/01/2020,11/28/2020,United States,All Sexes,All Ages,249570.0,2635214.0,249148.0,113943.0,6852.0,390588.0,NaN
1,12/02/2020,02/01/2020,11/28/2020,United States,All Sexes,Under 1 year,29.0,14843.0,139.0,4.0,15.0,179.0,NaN
2,12/02/2020,02/01/2020,11/28/2020,United States,All Sexes,0-17 years,127.0,26333.0,407.0,20.0,123.0,637.0,NaN
3,12/02/2020,02/01/2020,11/28/2020,United States,All Sexes,1-4 years,16.0,2768.0,91.0,2.0,42.0,147.0,NaN
4,12/02/2020,02/01/2020,11/28/2020,United States,All Sexes,5-14 years,44.0,4453.0,120.0,7.0,53.0,210.0,NaN


In [3]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2661 entries, 0 to 2660
Data columns (total 13 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Data as of                                2661 non-null   object 
 1   Start week                                2661 non-null   object 
 2   End Week                                  2661 non-null   object 
 3   State                                     2661 non-null   object 
 4   Sex                                       2661 non-null   object 
 5   Age group                                 2661 non-null   object 
 6   COVID-19 Deaths                           2271 non-null   float64
 7   Total Deaths                              2488 non-null   float64
 8   Pneumonia Deaths                          2181 non-null   float64
 9   Pneumonia and COVID-19 Deaths             2292 non-null   float64
 10  Influenza Deaths                    

In [4]:
missing_zero_values_table(raw_data)

,Zero Count,Na Count,% of Total,Zero + Na Count,% Zero + Na Count,Data Type
Footnote,0,1362,51.2,1362,51.2,object
Influenza Deaths,1406,765,28.7,2171,81.6,float64
Pneumonia Deaths,1046,480,18.0,1526,57.3,float64
"Pneumonia, Influenza, or COVID-19 Deaths",976,477,17.9,1453,54.6,float64
COVID-19 Deaths,1208,390,14.7,1598,60.1,float64
Pneumonia and COVID-19 Deaths,1368,369,13.9,1737,65.3,float64
Total Deaths,734,173,6.5,907,34.1,float64


Importantly, zero's do not mean that the data is missing, they simply mean that there were no deaths, so the test above is a little heavy handed.
Additionally, all of the footnotes are the message <b>"One or more data cells have counts between 1-9 and have been suppressed in accordance with NCHS confidentiality standards."</b>

Because of this, I will fill in all N/A values within rows that contain this footnote with <code>footnote_fill_value</code>

Any N/A values that are encountered without this footnote will be left blank

In [5]:
footnote_fill_value: int = 5
footnote_fill_text: str = "One or more data cells have counts between 1-9 and have been suppressed in accordance with NCHS confidentiality standards."
footnote_col: str = "Footnote"

In [6]:
numeric_columns: List[str] = raw_data.select_dtypes(include=['float64', 'int64']).columns

In [7]:
data: pd.DataFrame = raw_data.copy()
for col in numeric_columns:
    data.loc[(pd.isnull(data[col])), col] = footnote_fill_value

In [8]:
missing_zero_values_table(data)

,Zero Count,Na Count,% of Total,Zero + Na Count,% Zero + Na Count,Data Type
Footnote,0,1362,51.2,1362,51.2,object


So the fact that literally every Na value from every row that isn't the Footnote disappeared could mean two things:

- Our hospitals report all of the data they are responsible for (thats pretty cool)
    
- The only missing datapoints are values between 1 and 9   (this is speculation as a datapoint could be excluded from a row because of NCHS confidentiality and for some other reason, which means that the other missing value could be something else, but I don't think this is as likely considering <b>Every row with an Na value had that NCHS Confidentiality exclusion</b>

Because of this, I think that our initial assumption of filling in Na values with <code>footnote_fill_value</code> (initially 5.0) is reasonable

Moving on, lets actually breaking up this data by age

In [9]:
age_groups: List[str] = sorted(data['Age group'].unique())
print(age_groups)

['0-17 years', '1-4 years', '15-24 years', '18-29 years', '25-34 years', '30-49 years', '35-44 years', '45-54 years', '5-14 years', '50-64 years', '55-64 years', '65-74 years', '75-84 years', '85 years and over', 'All Ages', 'Under 1 year']


In [13]:
data_by_age: Dict[str, pd.DataFrame] = {}
for age_group in age_groups:
    data_by_age[age_group] = data[data['Age group'] == age_group]